# VK API - Базовые функции управления РК

In [2]:
import pandas as pd

from oauth2client.service_account import ServiceAccountCredentials
import requests
import vk_api
import gspread
import time
import numpy as np
import datetime


# Вывод всех операций в Jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [10]:
# Остапа
TOKEN = '111111111111111111111111111111111111111111111111111111111111'
ACCOUNT_ID = 2222222222 
CLIENT_ID = 33333333

In [11]:
params_base = {'account_id': ACCOUNT_ID}
session = vk_api.VkApi(token=TOKEN)

In [5]:
def result_to_df(lst):
    # https://ru.stackoverflow.com/questions/1413824/%D0%9F%D1%80%D0%B5%D0%BE%D0%B1%D1%80%D0%B0%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D1%8C-%D0%B2%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%BD%D1%8B%D0%B9-%D1%81%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C-%D0%B2-dataframe
    """
    Преобразуем lst (в виде списка МНОГОУРОВНЕВЫХ словарей) в Датафрейм
    Название колонки формируется АВТОМАТИЧЕСКИ - наименование ключа.
    Если словарь вложенный - составные ключи через точку
    Входные данные: Список словарей
    Выходные данные: Датафрейм
    """

    df = pd.DataFrame()
    for adds in lst:
        df = pd.concat([df, pd.json_normalize(adds)], ignore_index=True)
        
    return df

# =======================================================
# Campaigns
# =======================================================

# ads.getCampaigns - получить список кампаний

In [419]:
def getCampaigns():
    """
    https://vk.com/dev/ads.getCampaigns
    Получить список РКСоздать
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    params = params_base.copy() 
    
#     params['campaign_ids'] = "[1024594324]"
    
    result = session.method("ads.getCampaigns", params)
    df = result_to_df(result)
    # Преобразуем из UNIX-формата
    df['create_time'] = df['create_time'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    df['update_time'] = df['update_time'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    
    return result, df

result, df_result = getCampaigns()
df_result

,all_limit,day_limit,id,name,start_time,status,stop_time,type,create_time,goal_type,user_goal_type,is_cbo_enabled,update_time,views_limit
0,0,0,1024581970,РК-01,0,1,0,adaptive_ads,2022-08-02 10:40:28,0,0,False,2022-08-02 10:40:28,0.0
1,0,0,1024594187,test,0,1,0,normal,2022-08-03 03:55:09,0,0,False,2022-08-03 03:55:09,NaN
2,1000,1000,1024594191,test,1665349200,1,1665349200,normal,2022-08-03 03:57:42,0,0,False,2022-08-03 03:57:43,NaN
3,1000,1000,1024595966,РК-05,1665349200,0,1665435600,normal,2022-08-03 07:18:32,0,0,False,2022-08-03 07:18:34,NaN


# ads.createCampaigns - создать РК

In [200]:
def createCampaigns(params):
    """
    https://vk.com/dev/ads.createCampaigns
    Создать РК
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.createCampaigns", params)
    return result, result_to_df(result)

params = params_base.copy() 

Campaign_01 = {}
# Тип кампании
Campaign_01["type"] = "normal" 
# Название рекламной кампании
Campaign_01["name"] = "РК-05"
# дневной лимит в рублях
Campaign_01["day_limit"] = "1000"
# # Общий лимит в рублях
Campaign_01["all_limit"] = "1000"
# время запуска кампании в формате unixtime.
d = datetime.date(2022,10,10)
unixtime = str(int(time.mktime(d.timetuple())))
Campaign_01["start_time"] = unixtime  
# время остановки кампании в формате unixtime.
d = datetime.date(2022,10,11)
unixtime = str(int(time.mktime(d.timetuple())))
Campaign_01["stop_time"] = unixtime 
# статус рекламной кампании (1 — запущена, 0 — остановлена)
Campaign_01["status"] = 0

data = f"[{Campaign_01}]"
data = data.replace('\'', '"')   # Иначе - не получится
params["data"] = data

result, df_result = createCampaigns(params)
result

[{'id': 1024595966}]

# ads.deleteCampaigns - удаление (архивирование) РК

In [196]:
def deleteCampaigns(params):
    """
    https://vk.com/dev/ads.deleteCampaigns
    Удаление РК
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.deleteCampaigns", params)
    return result

params = params_base.copy() 
params['ids'] = "[1024594324]"

result = deleteCampaigns(params)
result

[600]

# =======================================================
# Ads
# =======================================================

# ads.getAds - возвращает список РО

In [217]:
def getAds(params):
    """
    https://vk.com/dev/ads.getAds
    возвращает список рекламных объявлений
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getAds", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 
# фильтр по рекламным кампаниям.
# params['campaign_ids'] = "[1024581970]"
result, df_result = getAds(params)
df_result

,id,campaign_id,status,approved,create_time,update_time,goal_type,cost_type,day_limit,all_limit,...,events_retargeting_groups,conversion_pixel_id,conversion_event_id,ad_format,cpc,ad_platform,publisher_platforms,publisher_platforms_auto,ad_platform_no_ad_network,has_campaign_budget_optimization
0,126933804,1024581970,0,0,1659436828,1659436828,2,0,0,0,...,[],0,0,11,2150,all,vk,0,1,False


# ads.getAdsLayout - возвращает описания внешнего вида РО

In [222]:
def getAdsLayout(params):
    """
    https://vk.com/dev/ads.getAdsLayout
    возвращает описания внешнего вида РО
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getAdsLayout", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 
# фильтр по рекламным кампаниям.
# params['campaign_ids'] = "[1024581970]"
result, df_result = getAdsLayout(params)
df_result

,id,campaign_id,goal_type,cost_type,age_restriction,title,description,link_type,link_url,link_domain,link_title,link_button,ad_format,preview_link,image_src,image_src_2x,icon_src,icon_src_2x,new_publisher_platforms
0,126933804,1024581970,2,0,0,Новый Заголовок,Удобный сервис для поиска выборов и кагндидатов,5,https://info.vybory.pro/,info.vybory.pro,Перейти на сайт,open_url,11,https://vk.com/ads?act=preview_ad&mid=93440559...,https://sun3-9.userapi.com/1cFl5TZWorZq00XiIJy...,https://sun3-9.userapi.com/ty8UVIL2OQ_QcHJMJpu...,https://sun3-13.userapi.com/y2BS_rDHAN2hn74ATz...,https://sun3-10.userapi.com/9aUP3XVODv6_NIMBxJ...,False


# ads.getAdsTargeting- возвращает параметры таргетинга РО

In [355]:
def getAdsTargeting(params):
    """
    https://vk.com/dev/ads.getAdsTargeting
    возвращает параметры таргетинга РО
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getAdsTargeting", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 
# фильтр по рекламным кампаниям.
params['campaign_ids'] = "[1024581970]"
result, df_result = getAdsTargeting(params)
df_result

,id,campaign_id,sex,age_from,age_to,geo_near,price_list_audience_type,count,geo_point_type
0,126933804,1024581970,1,18,19,"55.913569,37.573879,1000,-1,Ashan",0,240,regular


In [356]:
result

[{'id': '126933804',
  'campaign_id': '1024581970',
  'sex': '1',
  'age_from': '18',
  'age_to': '19',
  'geo_near': '55.913569,37.573879,1000,-1,Ashan',
  'price_list_audience_type': '0',
  'count': '240',
  'geo_point_type': 'regular'}]

# ads.updateAds - редактируем РО (меняем Таргетинг) - геотаргетинг

Минимальный размер аудитории объявления — 100 человек. Если меньше - ошибка.

In [336]:
def updateAds(params):
    """
    https://vk.com/dev/ads.updateAds
    Меняем  список рекламных объявлений
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.updateAds", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 

ad_01 = {}
# Идентификатор редактируемого объявления.
ad_01['ad_id'] = 126933804

# ad_01['title'] = "Заголовок РО новый"

# пол (0-любой, 1-женский, 2-мужской)
# ad_01['sex'] = 1

# Места для гео-таргетинга (Ashan)
ad_01['geo_near'] = "55.913569,37.573879,1000,Ashan"
# Тип мест гео-таргетинга. Возможные значения: regular — Регулярно бывает; home — Дом; work — Работа или учёба;
ad_01['geo_point_type'] = "regular"


data = f"[{ad_01}]"
data = data.replace('\'', '"')   # Иначе - не получится
params["data"] = data


result, df_result = updateAds(params)
result

[{'id': 126933804}]

# =======================================================
# Targeting - получение статистики по АУДИТОРИИ ТАРГЕТИНГА
# =======================================================

# ads.getTargetingStats- Возвращает размер целевой аудитории таргетинга, а также рекомендованные значения CPC и CPM

**ДВА ВАРИАНТА ВХОДНЫХ ПАРАМЕТРОВ:**
* id рекламного объявления
* Задаем пересень свойств аудитории (как при создании (изменении) РО)

In [337]:
def getTargetingStats(params):
    """
    https://vk.com/dev/ads.getTargetingStats
    Возвращает размер целевой аудитории таргетинга, а также рекомендованные значения CPC и CPM
    
    ДВА ВАРИАНТА ВХОДНЫХ ПАРАМЕТРОВ:
    - id рекламного объявления
    - Задаем пересень свойств аудитории (как при создании (изменении) РО)
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getTargetingStats", params)
#     df = result_to_df(result)
    
    return result

params = params_base.copy() 
# Идентификатор редактируемого объявления.
params['ad_id'] = 126933804
# Формат объявления (1 - изображение и текст)
params['ad_format'] = 1
# Рекламные площадки, на которых будет показываться объявление (1 — только ВКонтакте)
params['ad_platform'] = 1
# Ссылка на рекламируемый объект
params['link_url'] = "https://info.vybory.pro/"
# Домен рекламируемого объекта.
params['link_domain'] = "info.vybory.pro"
# Точность вывода:   Если установлен, то дополнительно в ответе будут рекомендованные 
# значения CPC и CPM для достижения 5, 10, 15... 95% охвата
params['need_precise'] = "1"

result = getTargetingStats(params)
result

{'audience_count': 240,
 'recommended_cpc_50': '79.50',
 'recommended_cpc_70': '79.50',
 'recommended_cpc_90': '90.75',
 'recommended_cpm_50': '1.20',
 'recommended_cpm_70': '1.20',
 'recommended_cpm_90': '1.95',
 'recommended_cpm': '1.20',
 'recommended_cpc': '79.50',
 'total_alive_audience': 84,
 'recommended_cpc_precise': {'5': '6.00',
  '10': '6.00',
  '15': '6.00',
  '20': '6.00',
  '25': '63.00',
  '30': '63.00',
  '35': '63.00',
  '40': '63.00',
  '45': '63.00',
  '50': '79.50',
  '55': '79.50',
  '60': '79.50',
  '65': '79.50',
  '70': '79.50',
  '75': '90.75',
  '80': '90.75',
  '85': '90.75',
  '90': '90.75',
  '95': '90.75'},
 'recommended_cpm_precise': {'5': '1.20',
  '10': '1.20',
  '15': '1.20',
  '20': '1.20',
  '25': '1.20',
  '30': '1.20',
  '35': '1.20',
  '40': '1.20',
  '45': '1.20',
  '50': '1.20',
  '55': '1.20',
  '60': '1.20',
  '65': '1.20',
  '70': '1.20',
  '75': '1.50',
  '80': '1.65',
  '85': '1.75',
  '90': '1.95',
  '95': '2.20'}}

# =======================================================
# TargetGroup - РЕтаргетинг - Аудитории (сегменты)
# =======================================================

# ads.getTargetGroups- Возвращает список аудиторий ретаргетинга

In [432]:
def getTargetGroups(params):
    """
    https://vk.com/dev/ads.getTargetGroups
    Возвращает список аудиторий ретаргетинга
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getTargetGroups", params)
    df = result_to_df(result)
    
    return result, df

params = params_base.copy() 

result, df_result = getTargetGroups(params)
df_result

,id,name,last_updated,audience_count,lifetime,is_audience,is_shared,file_source,api_source,lookalike_source
0,48652446,phone_500,1659636731,970,720,True,False,True,True,False
1,48652574,phone_100000,1659594635,97000,720,True,False,True,False,False
2,48665906,Null segment,1659617143,520,720,True,False,False,True,False
3,48673250,Lookalike-аудитория 4 авг,0,0,720,True,False,False,False,False
4,48682492,5 августа,1659683209,0,720,True,False,False,True,False


# ads.createTargetGroup - Создаёт аудиторию

Создаёт аудиторию для ретаргетинга рекламных объявлений на пользователей, которые посетили сайт рекламодателя (просмотрели информации о товаре, зарегистрировались и т.д.).

В результате выполнения метода возвращается идентификатор созданной аудитории.

In [426]:
def createTargetGroup(params):
    """
    https://vk.com/dev/ads.createTargetGroup
    Создаёт аудиторию для ретаргетинга рекламных объявлений на пользователей, которые посетили сайт рекламодателя 
    (просмотрели информации о товаре, зарегистрировались и т.д.)
    
    Остап: можно создать "пустую" аудиторию, а затем с помощью метода "importTargetContacts" - добавить туда контакты!!!
    
    В результате выполнения метода возвращается идентификатор созданной аудитории.
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.createTargetGroup", params)
    
    return result

params = params_base.copy()
# Наименование АУДИТОРИИ
params['name'] = "5 августа"

result = createTargetGroup(params)
result

{'id': 48682492}

# ads.removeTargetContacts - исключение контактов из Аудитории

In [381]:
def removeTargetContacts(params):
    """
    https://vk.com/dev/ads.removeTargetContacts
    Исключение контактов из Аудитории
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.removeTargetContacts", params)
    
    return result

params = params_base.copy() 
# id аудитории (сегмента)
params['target_group_id'] = "48652446"
params['contacts'] = '79602636784,79222537777,79036614840,79109706112,79029236330,79164524626,79277048129,79218794833,79161515182,79606903778,79104169717,79685044040,79635740605,79151017298'

result = removeTargetContacts(params)
result

{'result': 1}

# ads.importTargetContacts - Импортирует список контактов

In [429]:
phone_list_100000

['79100053326',
 '79990340663',
 '79850681908',
 '79166895758',
 '79263423948',
 '79639990171',
 '79112588846',
 '79250774711',
 '79060553061',
 '79262831055',
 '79633159646',
 '79684520704',
 '79067274268',
 '79259085179',
 '79146533082',
 '79617133123',
 '79874351899',
 '79202917691',
 '79500057463',
 '79875347598',
 '79217885499',
 '79162884525',
 '79628672888',
 '79250351945',
 '79817715986',
 '79165243123',
 '79112679369',
 '79261337150',
 '79372392192',
 '79313396948',
 '79197646286',
 '79112668573',
 '79654530303',
 '79169166050',
 '79119623304',
 '79255693169',
 '79245553838',
 '79262190440',
 '79294114987',
 '79266977398',
 '79150557977',
 '79776288412',
 '79832691970',
 '79054397888',
 '79859369368',
 '79104017010',
 '79067954424',
 '79897079909',
 '79651999993',
 '79806520029',
 '79115848223',
 '79118813399',
 '79777034970',
 '79217734214',
 '79315892939',
 '79602533233',
 '79898824141',
 '79269828718',
 '79105925837',
 '79832227798',
 '79059083281',
 '79205964777',
 '796897

In [431]:
def importTargetContacts(params):
    """
    https://vk.com/dev/ads.importTargetContacts
    Импортирует список контактов рекламодателя для учета зарегистрированных 
    во ВКонтакте пользователей в аудитории ретаргетинга.
    Максимально допустимое количество контактов, импортируемых с помощью одного запроса — 1000.
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.importTargetContacts", params)
    
    return result

params = params_base.copy() 
# id аудитории (сегмента)
params['target_group_id'] = "48682492"
params['contacts'] = phone_500_end

result = importTargetContacts(params)
result

500

# ads.deleteTargetGroup - Удаляет аудиторию

In [368]:
def deleteTargetGroup(params):
    """
    https://vk.com/dev/ads.deleteTargetGroup
    Удаляет аудиторию
    В случае успеха метод возвратит 1
    """
    
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.deleteTargetGroup", params)
    
    return result

params = params_base.copy() 
params['target_group_id'] = "48665288"

result = deleteTargetGroup(params)
result

1

# =======================================================
# Lookalike (поиск похожей аудитории) - Аудитории (сегменты)
# =======================================================

# ads.getLookalikeRequests - Возвращает список запросов на создание Lookalike аудитории

In [12]:
def getLookalikeRequests(params):
    """
    https://vk.com/dev/ads.getLookalikeRequests
    Возвращает список запросов Lookalike
    
    status — текущий статус запроса, может принимать следующие значения: 
    - search_in_progress 
    - search_done
    - search_failed
    - save_in_progress 
    - save_done
    - save_failed
    """
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.getLookalikeRequests", params)
    df = result_to_df(result['items'])
    return result, df

params = params_base.copy() 
# Список идентификаторов запрашиваемых запросов через запятую. 
# Если этот параметр пуст, возвращаться будут все запросы
params['requests_ids'] = "72839"

result, result_df = getLookalikeRequests(params)
result_df

,id,create_time,update_time,status,source_type,source_retargeting_group_id,source_name,audience_count
0,72839,1659621139,1659622802,search_done,retargeting_group,48652574,phone_100000,56000


In [ ]:
{'count': 1,
 'items': [{'id': 72839,
   'create_time': 1659621139,
   'update_time': 1659621139,
   'status': 'search_in_progress',
   'source_type': 'retargeting_group',
   'source_retargeting_group_id': 48652574,
   'source_name': 'phone_100000',
   'audience_count': 56000}]}

In [417]:
result

{'count': 1,
 'items': [{'id': 72839,
   'create_time': 1659621139,
   'update_time': 1659622802,
   'status': 'search_done',
   'source_type': 'retargeting_group',
   'source_retargeting_group_id': 48652574,
   'source_name': 'phone_100000',
   'audience_count': 56000}]}

# ads.createLookalikeReques - Создаёт запрос на поиск Аудитории Lookalike

In [398]:
def createLookalikeRequest(params):
    """
    https://vk.com/dev/ads.createLookalikeRequest
    Создаёт запрос на поиск похожей аудитории.
    Поиск похожей аудитории займёт от нескольких минут до нескольких часов в зависимости от нагрузки. 
    За состоянием поиска можно следить с помощью ads.getLookalikeRequests.
    После завершения поиска Вы сможете выбрать точность/охват и сохранить аудиторию через ads.saveLookalikeRequestResult.
    Минимальный размер исходной аудитории –  1000 (раньше - 300) активных пользователей.
    Если исходная аудитория маленькая, выдает ответ. типа:
    --- [632] Given audience is too small: Минимальный охват исходной аудитории — 1000 пользователей, проявлявших активность 
    за последние 2 недели. Активных пользователей в выбранной аудитории: 310 человек --- 
    Максимальный размер исходной аудитории –  1000000 активных пользователей.
    Единовременно разрешён только один незавершённый запрос на поиск похожей аудитории для конкретного источника.
    Лимит количества запросов –  5 запросов в сутки на кабинет (клиента в случае агентства).
    """
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.createLookalikeRequest", params)
    return result

params = params_base.copy() 

# Тип источника исходной аудитории. На данный момент может принимать единственное значение retargeting_group
params['source_type'] = 'retargeting_group'
# id аудитории (сегмента)
params['retargeting_group_id'] = '48652574'   # Исходная аудитория - 100 тыс

result = createLookalikeRequest(params)
result

{'request_id': 72839}

# ads.saveLookalikeRequestResult - Сохраняет результат Lookalike

In [414]:
def saveLookalikeRequestResult(params):
    """
    https://vk.com/dev/ads.saveLookalikeRequestResult
    Сохраняет результат поиска похожей аудитории.
    Сохранение выполняется в группу ретаргетинга по указанному размеру аудитории.
    Доступно только для запросов с одним из статусов search_done, save_done, и save_failed. Сохранение аудитории происходит некоторое время, за прогрессом можно следить через статус запроса (ads.getLookalikeRequests).
    Разрешается сохранять результат конкретного запроса на поиск похожей аудитории одновременно только в одну аудиторию ретаргетинга
    """
    time.sleep(0.5)  # Обязательно нужна задержка.Иначе VK-игнорирует запрос (Можно 2 раза в секунду)
    
    result = session.method("ads.saveLookalikeRequestResult", params)
    return result

params = params_base.copy() 

# Идентификатор запроса на поиск похожей аудитории. Получить список всех запросов на поиск похожей аудитории 
# можно с помощью ads.getLookalikeRequests
params['request_id'] = '72839'
# id аудитории (сегмента)
params['retargeting_group_id'] = '48673250'  


result = saveLookalikeRequestResult(params)
result

ApiError: [100] One of the parameters specified was missing or invalid: level is undefined